# NLP: INT3406E_41 (2022) - PGS.TS: Nguyen Phuong Thai

---


PROGRAMMING ASSIGNMENT 1: WORD SIMILARITY AND SEMANTIC RELATION CLASSIFICATION
#Vũ Hoàng Long 19020036
#Trần Long Dũng 19021321

In [ ]:
!pip install gdown
!git clone https://github.com/NLP-Projects/Word-Similarity.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'Word-Similarity'...
remote: Enumerating objects: 196, done.
remote: Total 196 (delta 0), reused 0 (delta 0), pack-reused 196
Receiving objects: 100% (196/196), 5.81 MiB | 10.64 MiB/s, done.
Resolving deltas: 100% (77/77), done.


# EX1: Cosine similarity:
Given pre-trained embeddings of Vietnamese words, implement a function for calculating cosine similarity between word pairs. Test your program using word pairs in ViSim-400 dataset (in directory Datasets/ViSim-400).

**Dowload pretrained Word2vec Dataset:**
Link download pre-trained word2vec model 
https://drive.google.com/open?id=1z1IDKaZuJXw5g7Yebr1GDq2UfvVR_aGx

In [ ]:
!gdown https://drive.google.com/uc?id=1z1IDKaZuJXw5g7Yebr1GDq2UfvVR_aGx

Downloading...
From: https://drive.google.com/uc?id=1z1IDKaZuJXw5g7Yebr1GDq2UfvVR_aGx
To: /content/W2V_150.txt
100% 119M/119M [00:00<00:00, 180MB/s]


**Extract pretrained word2vec data into a Dictionary for searching embedded vectors by key work**

In [ ]:
import csv

vec_data_file = '/content/W2V_150.txt'
def word2vec_dictionary(data_file):
  i = 0
  emb_vec_dict = {}
  with open(data_file, 'r') as fd:
      reader = csv.reader(fd)
      for row in reader:
        split_data = row[0].split(' ')
        word = split_data[0]
        emb_vec = []
        for v in split_data[2:-1]:
          emb_vec.append(float(v))
        emb_vec_dict[word] = emb_vec
  return emb_vec_dict
word2vec = word2vec_dictionary(vec_data_file)

In [ ]:
print(len(word2vec['chó']), len(word2vec['mèo']))

150 150


**Define the cosine_similarity function for calculating the cosine similarity between 2 vecs:**

In [ ]:
import numpy as np
from scipy import spatial

a = [1,2,3,4]
b = [5,6,7,8]
def cosine_similarity_cal(v1, v2):
  return (2 - spatial.distance.cosine(v1, v2))/2
print(cosine_similarity_cal(a,b))

0.9844319658134832


**Calculate the cosine similarity between 2 word by vec:**

In [ ]:
data_file = '/content/Word-Similarity/datasets/ViSim-400/Visim-400.txt'
#data = f = open(data_file, "r")
import csv
i = 0
n_compair = 10
with open(data_file, 'r') as fd:
    reader = csv.reader(fd)
    #pair_words = []
    for row in reader:
      split_data = row[0].split('\t')
      word1 = split_data[0]
      word2 = split_data[1]
      try:
        vec1 = np.array(word2vec[word1])
        vec2 = np.array(word2vec[word2])
        print("Pair-Word: {} - {}, Cosine-similarity: {}".format(word1, word2, cosine_similarity_cal(vec1, vec2)))
      except:
        print('This compair word was not found in dictionary!')
      i+=1
      if i == n_compair:
        break

This compair word was not found in dictionary!
Pair-Word: biến - ngập, Cosine-similarity: 0.497543830265165
Pair-Word: nhà_thi_đấu - nhà, Cosine-similarity: 0.5412615916460588
Pair-Word: động - tĩnh, Cosine-similarity: 0.6385429799341388
Pair-Word: khuyết - ưu, Cosine-similarity: 0.5883993141781336
This compair word was not found in dictionary!
Pair-Word: thủ_pháp - biện_pháp, Cosine-similarity: 0.7011830645971531
Pair-Word: kết_duyên - thành_hôn, Cosine-similarity: 0.7315042010070361
Pair-Word: cấp_tiến - bảo_thủ, Cosine-similarity: 0.6284735044498062
Pair-Word: nước_lớn - nguy_hiểm, Cosine-similarity: 0.5925960112010581


# EX2: K-nearest words: Given a word w, find k most-similar words of w using the function implemented in question 1.

**Find k nearest text function:**

In [ ]:
def k_nearest_neighbor(w, v, k = 1):
  similar_vec = [] #create a similar vec list for choosing a nearest vector
  map_v2w = {} #create a dictionary for mapping the vector to word after find the k nearest
  k_nearest_word = [] #create a k-nearest list word for return the near rest word list 
  for i in range(len(v)):
    w_vec = word2vec[w] #search the embedding vector of the word
    word_vec = word2vec[v[i]] #search the embedding vector of the word
    csc = cosine_similarity_cal(w_vec, word_vec) #calculate the cosine similarity
    similar_vec.append(csc) #append similar score to list for searching the nearest
    map_v2w[csc] = v[i] #adding the key: vector and value: word to dictionary for quering the word which is nearest
  similar_vec.sort() #sorting by ascending order
  k_nearest_vec = similar_vec[-k:] #quering the k-nearest vector
  for vec in k_nearest_vec:
    word = map_v2w[vec] #mapping the k-nearest vector to word
    k_nearest_word.append(word)
  return k_nearest_word

#Testing function
k = 3
w = "báo_đốm"
v = ["nước_lớn", "sông", "nhà_thi_đấu", "kết_duyên", "mèo", "thủy", "nguy_hiểm", "sóng", "hổ", "sư_tử"]
print("Input w word: {}".format(w))
print("{} nearest:".format(k), k_nearest_neighbor(w, v, 3))

Input w word: báo_đốm
3 nearest: ['sư_tử', 'hổ', 'mèo']


# EX3: Synonym-antonym classification: Implement a Logistic Regression classifier or a Multi-layer Perceptron classifier for distinguishing synonyms and antonyms.

**Extract 301 Vector: (150 each word) and Sim Score (1 each pair-word) of Synonym and Antonym dataset**

In [ ]:
import pandas as pd

ant_file = '/content/Word-Similarity/antonym-synonym set/Antonym_vietnamese.txt'
syn_file = '/content/Word-Similarity/antonym-synonym set/Synonym_vietnamese.txt'

css = []
type_of_word = []
dict_dataset = {}
dict_lists = {}
for i in range(1,301):
  dict_lists['l'+str(i)] = []
with open(ant_file, 'r') as fd:
    for row in fd: 
      split_word = row.split(' ')
      try: #try if dictionary have that word
        word1 = split_word[0].strip()
        word2 = split_word[1].strip()
        vec1 = np.array(word2vec[word1])
        vec2 = np.array(word2vec[word2])
        vec12 = np.concatenate((vec1, vec2))
        css.append(cosine_similarity_cal(vec1, vec2))
        type_of_word.append(0)
        for i in range(300):
            dict_lists['l'+str(i+1)].append(vec12[i])
      except:
        pass
with open(syn_file, 'r') as fd:
    for row in fd:
      split_word = row.split(' ')
      try: #try if dictionary have that word
        word1 = split_word[0].strip()
        word2 = split_word[1].strip()
        vec1 = word2vec[word1]
        vec2 = word2vec[word2]
        vec12 = np.concatenate((vec1, vec2))
        css.append(cosine_similarity_cal(vec1, vec2))
        type_of_word.append(1)
        for i in range(300):
            dict_lists['l'+str(i+1)].append(vec12[i])
      except:
        pass
dict_dataset['CSS'] = css
dict_dataset['Type'] = type_of_word
for i in range(300):
  dict_dataset['vec'+str(i+1)] = dict_lists['l'+str(i+1)]
df = pd.DataFrame(dict_dataset)
df = df.reindex(np.random.permutation(df.index))

df_no_type = df.drop('Type', axis=1)
features = df_no_type.to_numpy()
labels = df['Type'].to_numpy()

In [ ]:
print(df)

           CSS  Type      vec1      vec2      vec3      vec4      vec5  \
7965  0.649490     1 -0.969126  2.503197  1.606285  1.267443  0.704963   
750   0.652996     0  0.032384  0.886631 -2.745771 -2.501953  3.117980   
6738  0.548607     1  0.141208  0.497018 -0.325516 -1.347731 -0.125896   
5321  0.884776     1  0.613247  2.044406  0.722219  0.670574  0.383656   
2690  0.807124     1  0.819285  0.143123  1.499029 -2.458985  1.112694   
...        ...   ...       ...       ...       ...       ...       ...   
3660  0.669240     1 -1.534978  0.975442 -1.254490  1.011634  1.062638   
6505  0.718027     1  1.242174  0.615960  0.674266  0.535518 -2.756795   
3890  0.766592     1  2.369785  1.884284  1.460598  0.756433  1.119153   
7059  0.685703     1  3.048424  3.572027  1.315129  0.211011  1.833323   
5409  0.557399     1  1.249571 -0.058011  0.383467  1.799968 -0.748270   

          vec6      vec7      vec8  ...    vec291    vec292    vec293  \
7965  2.762822  0.602556  0.602971  ..

In [ ]:
df.to_csv('Word_pair_by_Vec_Data.csv', sep='\t')

**Split data for training and testing**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(features, labels, test_size=0.30)
print('Training records:', Y_train.size)
print('Val records:', Y_val.size)

Training records: 5598
Val records: 2400


**Testing training with sklearn nn MLP classifier**

In [ ]:
# Import MLPClassifer 
from sklearn.neural_network import MLPClassifier

# Create model object
clf = MLPClassifier(hidden_layer_sizes=(64,32),
                    random_state=1,
                    verbose=True,
                    learning_rate_init=0.01)

# Fit data onto the model
clf.fit(X_train,Y_train)

Iteration 1, loss = 0.40900404
Iteration 2, loss = 0.20151944
Iteration 3, loss = 0.09910788
Iteration 4, loss = 0.04263549
Iteration 5, loss = 0.02712714
Iteration 6, loss = 0.01565339
Iteration 7, loss = 0.01673839
Iteration 8, loss = 0.01415252
Iteration 9, loss = 0.02768228
Iteration 10, loss = 0.04415880
Iteration 11, loss = 0.03170392
Iteration 12, loss = 0.02031861
Iteration 13, loss = 0.01063132
Iteration 14, loss = 0.00576693
Iteration 15, loss = 0.00480739
Iteration 16, loss = 0.00489751
Iteration 17, loss = 0.00257582
Iteration 18, loss = 0.00170721
Iteration 19, loss = 0.00140536
Iteration 20, loss = 0.00111618
Iteration 21, loss = 0.00083685
Iteration 22, loss = 0.00070379
Iteration 23, loss = 0.00076875
Iteration 24, loss = 0.00061915
Iteration 25, loss = 0.00084165
Iteration 26, loss = 0.00083560
Iteration 27, loss = 0.00060354
Iteration 28, loss = 0.00077761
Iteration 29, loss = 0.00129941
Iteration 30, loss = 0.00095950
Iteration 31, loss = 0.00088255
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(64, 32), learning_rate_init=0.01,
              random_state=1, verbose=True)

**TEST THE MODEL WITH VISIM400 and print OUTPUT**

In [ ]:
from sklearn import metrics
test_file = '/content/Word-Similarity/datasets/ViCon-400/400_noun_pairs.txt'
#data = f = open(data_file, "r")
import csv
k = 0
syn_ant = {
    "ANT": 0,
    "SYN": 1
}
n_compair = 1
test_data = []
vec = []
css = []
type_of_word = []
dict_testset = {}
dict_lists = {}
p_word1 = []
p_word2 = []
for i in range(1,301):
  dict_lists['l'+str(i)] = []
with open(test_file, 'r') as fd:
      for row in fd:
        if k >= 1:
          split_data = row.split('\t')
          try: #try if dictionary have that word
            word1 = split_data[0].strip()
            word2 = split_data[1].strip()
            types = split_data[2].strip()
            #print(word1, word2, syn_ant[types])
            vec1 = np.array(word2vec[word1])
            vec2 = np.array(word2vec[word2])
            type_of_word.append(syn_ant[types])
            vec12 = np.concatenate((vec1, vec2))
            css.append(cosine_similarity_cal(vec1, vec2))
            p_word1.append(word1)
            p_word2.append(word2)
            for i in range(300):
              dict_lists['l'+str(i+1)].append(vec12[i])
          except:
            pass
        k+=1
dict_testset['Word1'] = p_word1
dict_testset['Word2'] = p_word2
dict_testset['CSS'] = css
dict_testset['Type'] = type_of_word
for i in range(300):
  dict_testset['vec'+str(i+1)] = dict_lists['l'+str(i+1)]
df = pd.DataFrame(dict_testset)
df_only_input = df.drop(['Word1','Word2','Type'], axis=1)
# create test data
test = df_only_input.to_numpy()
test_labels = df['Type'].to_numpy()
# prediction
predicted_y = clf.predict(test)
#print(X_val)
#print(predicted_y)
#Print output
print("Accuracy:")
print("___________________________________________________________________")
print(metrics.classification_report(test_labels, predicted_y))
output_dict = {"Word1" : p_word1,
               "Word2" : p_word2,
               "Cosine Similarity": css,
               "Predicted": predicted_y,
               "True Value": type_of_word
               }
out_test = pd.DataFrame(output_dict)
print("____________________________________________________________________")
print("1 is Synonym and 0 is Antonym, Output test Visualize:")
print("____________________________________________________________________")
print(out_test)

Accuracy:
___________________________________________________________________
              precision    recall  f1-score   support

           0       0.98      0.94      0.96       172
           1       0.94      0.98      0.96       154

    accuracy                           0.96       326
   macro avg       0.96      0.96      0.96       326
weighted avg       0.96      0.96      0.96       326

____________________________________________________________________
1 is Synonym and 0 is Antonym, Output test Visualize:
____________________________________________________________________
          Word1     Word2  Cosine Similarity  Predicted  True Value
0     khoái_lạc   nỗi_đau           0.622883          1           0
1       yếu_kém  sức_mạnh           0.585686          0           0
2    thanh_danh    ô_nhục           0.544493          0           0
3           dây       dọc           0.570831          1           1
4            ba        me           0.475859          0        

**Implement the MLP from scratch**

In [ ]:
import time
class MultiLayerPerceptron():
  def __init__(self, sizes, epochs=10, l_rate=0.001):
    self.sizes = sizes
    self.epochs = epochs
    self.l_rate = l_rate

    self.params = self.initialization()
  
  def sigmoid(self, x, der=False):
    if der:
      return (np.exp(-x))/((np.exp(-x)+1)**2)
    return 1/(1+np.exp(-x))
  
  def softmax(self, x, der=False):
        exps = np.exp(x - x.max())
        if der:
            return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))
        return exps / np.sum(exps, axis=0)

  def ReLu(self, x, der=False):
    if der:
      return np.heaviside(x, 1)
    return np.maximum(x, 0)
    
  def initialization(self):
    input_layer = self.sizes[0]
    hidden_1 = self.sizes[1]
    hidden_2 = self.sizes[2]
    output_layer = self.sizes[3]

    params = {
        'W1':np.random.rand(hidden_1, input_layer) * np.sqrt(1. / hidden_1),
        'W2':np.random.rand(hidden_2, hidden_1) * np.sqrt(1. / hidden_2),
        'W3':np.random.rand(output_layer, hidden_2) * np.sqrt(1. / output_layer)
    }

    return params

  def forward_pass(self, x_train):
    params = self.params

    params['A0'] = x_train

    params['Z1'] = np.dot(params["W1"], params['A0'])
    params['A1'] = self.ReLu(params['Z1'])

    params['Z2'] = np.dot(params["W2"], params['A1'])
    params['A2'] = self.ReLu(params['Z2'])

    params['Z3'] = np.dot(params["W3"], params['A2'])
    params['A3'] = self.sigmoid(params['Z3'])

    return params['A3']

  def backward_pass(self, y_train, output):
    params = self.params
    change_w = {}

    # Calculate W3 update
    error = 2 * (output - y_train) / output.shape[0] * self.softmax(params['Z3'], der=True)
    change_w['W3'] = np.outer(error, params['A2'])

    # Calculate W2 update
    error = np.dot(params['W3'].T, error) * self.ReLu(params['Z2'], der=True)
    change_w['W2'] = np.outer(error, params['A1'])

    # Calculate W1 update
    error = np.dot(params['W2'].T, error) * self.ReLu(params['Z1'], der=True)
    change_w['W1'] = np.outer(error, params['A0'])

    return change_w

  def update_network_parameters(self, changes_to_w):
    for key, value in changes_to_w.items():
            self.params[key] -= self.l_rate * value
  
  def compute_accuracy(self, x_val, y_val):
    predictions = []
    for x, y in zip(x_val, y_val):
      output = self.forward_pass(x)
      pred = np.argmax(output)
      predictions.append(pred == np.argmax(y))
    
    return np.mean(predictions)
  
  def train(self, x_train, y_train, x_val, y_val):
    start_time = time.time()
    for iteration in range(self.epochs):
      for x, y in zip(x_train, y_train):
        output = self.forward_pass(x)
        changes_to_w = self.backward_pass(y, output)
        self.update_network_parameters(changes_to_w)
      
      accuracy = self.compute_accuracy(x_val, y_val)
      print('Epoch: {0}, Time spent: {1:.2f}s, Accuracy: {2:.2f}%'.format(
          iteration+1, time.time() - start_time, accuracy * 100
      ))
    
  def predict(self, x_test):
    predictions = []
    for x in (x_test):
      output = self.forward_pass(x)
      predictions.append(np.argmax(output))
    return predictions


**Training . . .**

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

mlp = MultiLayerPerceptron(sizes=[301, 40, 10, 2], epochs = 100, l_rate=0.005)
mlp.train(X_train, Y_train, X_val, Y_val)


Epoch: 1, Time spent: 1.06s, Accuracy: 35.17%
Epoch: 2, Time spent: 2.14s, Accuracy: 72.42%
Epoch: 3, Time spent: 3.20s, Accuracy: 66.12%
Epoch: 4, Time spent: 4.26s, Accuracy: 47.54%
Epoch: 5, Time spent: 5.30s, Accuracy: 48.83%
Epoch: 6, Time spent: 6.34s, Accuracy: 47.58%
Epoch: 7, Time spent: 7.49s, Accuracy: 48.38%
Epoch: 8, Time spent: 8.53s, Accuracy: 49.50%
Epoch: 9, Time spent: 9.57s, Accuracy: 50.25%
Epoch: 10, Time spent: 10.62s, Accuracy: 50.71%
Epoch: 11, Time spent: 11.68s, Accuracy: 57.00%
Epoch: 12, Time spent: 12.74s, Accuracy: 56.46%
Epoch: 13, Time spent: 13.82s, Accuracy: 55.21%
Epoch: 14, Time spent: 14.95s, Accuracy: 54.75%
Epoch: 15, Time spent: 16.01s, Accuracy: 53.67%
Epoch: 16, Time spent: 17.09s, Accuracy: 53.96%
Epoch: 17, Time spent: 18.16s, Accuracy: 53.33%
Epoch: 18, Time spent: 19.21s, Accuracy: 51.54%
Epoch: 19, Time spent: 20.31s, Accuracy: 51.21%
Epoch: 20, Time spent: 21.34s, Accuracy: 50.33%
Epoch: 21, Time spent: 22.40s, Accuracy: 50.33%
Epoch: 22,

**Predict and compute accuracy:**

In [ ]:
predict = mlp.predict(test[:10])
output_dict = {"Word1" : p_word1[:10],
               "Word2" : p_word2[:10],
               "Cosine Similarity": css[:10],
               "Predicted": predict,
               "True Value": type_of_word[:10]
               }
out_test = pd.DataFrame(output_dict)
print(metrics.classification_report(out_test['True Value'], predict))
print("____________________________________________________________________")
print("1 is Synonym and 0 is Antonym, Output test Visualize:")
print("____________________________________________________________________")
print(out_test)

              precision    recall  f1-score   support

           0       0.67      0.50      0.57         8
           1       0.00      0.00      0.00         2

    accuracy                           0.40        10
   macro avg       0.33      0.25      0.29        10
weighted avg       0.53      0.40      0.46        10

____________________________________________________________________
1 is Synonym and 0 is Antonym, Output test Visualize:
____________________________________________________________________
          Word1      Word2  Cosine Similarity  Predicted  True Value
0     khoái_lạc    nỗi_đau           0.622883          0           0
1       yếu_kém   sức_mạnh           0.585686          0           0
2    thanh_danh     ô_nhục           0.544493          0           0
3           dây        dọc           0.570831          0           1
4            ba         me           0.475859          1           0
5          ngày    ban_đêm           0.576550          1           

**Result is very bad!!! =>>>>> Underfitting: Model Cannot Fitting**